In [8]:
import pandas
# import urllib.request
# requests works better...I'm not sure why
#but my encoding problem is gone.
#See: https://stackoverflow.com/questions/47419104/unicodedecodeerror-utf-8-codec-cant-decode-python3
import requests
import json
import geojson
import folium

In [46]:
#GDELT GeoJson Endpoint
url = "https://api.gdeltproject.org/api/v1/gkg_geojson?MAXROWS=3"

#Issue request to URL
request = requests.get(url)
print(request.status_code)
if request.status_code == 200: data = request.json()
print(type(data))

200
<class 'dict'>


In [ ]:
#another thing to try
#via https://www.haykranen.nl/2016/02/13/handling-complex-nested-dicts-in-python/

class DictQuery(dict):
    def get(self, path, default = None):
        keys = path.split("/")
        val = None

        for key in keys:
            if val:
                if isinstance(val, list):
                    val = [ v.get(key, default) if v else None for v in val]
                else:
                    val = val.get(key, default)
            else:
                val = dict.get(self, key, default)

            if not val:
                break;

        return val

In [7]:
# def flatten(indict, pre=None):
#     pre = pre[:] if pre else []
#     if isinstance(indict, dict):
#         for key, value in indict.items():
#             if isinstance(value, dict):
#                 for d in flatten(value, [key] + pre):
#                     yield d
#                 elif isinstance(value, list) or isinstance(value, tuple):
#                     for v in value:
#                         for d in flatten(v, [key] + pre):
#                             yield d
#                 else:
#                     yield pre + [key, value]
#             else:
#                 yield indict

    
#from: https://stackoverflow.com/questions/12507206/python-recommended-way-to-walk-complex-dictionary-structures-imported-from-json    
def dict_generator(indict, pre=None):
    pre = pre[:] if pre else []
    if isinstance(indict, dict):
        for key, value in indict.items():
            if isinstance(value, dict):
                for d in dict_generator(value, [key] + pre):
                    yield d
            elif isinstance(value, list) or isinstance(value, tuple):
                for v in value:
                    for d in dict_generator(v, [key] + pre):
                        yield d
            else:
                yield pre + [key, value]
    else:
        yield indict
        
for i in dict_generator(data):
    print(i)

['type', 'FeatureCollection']
['features', 'type', 'Feature']
['geometry', 'features', 'type', 'Point']
55.28
25.2522
['properties', 'features', 'urlpubtimedate', '2018-02-01T13:45:00Z']
['properties', 'features', 'name', 'Dubai, Dubayy, United Arab Emirates']
['properties', 'features', 'urltone', 3.03]
['properties', 'features', 'url', 'https://www.arabfinance.com/ar/news/details/world-companies/421816']
['properties', 'features', 'mentionedthemes', ';TAX_WORLDLANGUAGES_ISLAMI;TAX_ETHNICITY_ARAB;TAX_ETHNICITY_ARAB;TAX_ETHNICITY_ARAB;']
['features', 'type', 'Feature']
['geometry', 'features', 'type', 'Point']
55.28
25.2522
['properties', 'features', 'urlpubtimedate', '2018-02-01T13:45:00Z']
['properties', 'features', 'name', 'Dubai, Dubayy, United Arab Emirates']
['properties', 'features', 'urltone', 3.03]
['properties', 'features', 'url', 'https://www.arabfinance.com/ar/news/details/world-companies/421816']
['properties', 'features', 'mentionedthemes', ';EPU_CATS_REGULATION;EPU_CATS_F

In [54]:
# print(data['features'])

flat_dictionary = {}
for feature in data['features']:
    for key, value in feature.items():
      if type(feature[key]) == dict:
        for k, v in feature[key].items():
            if k in flat_dictionary.keys():
                flat_dictionary[k].append(feature[key][k])
            else:
                flat_dictionary[k] = [v]

print(flat_dictionary)
print("---------")
for k,v in flat_dictionary.items(): 
    print(k,v)
    
    
    
df_features = pandas.DataFrame.from_dict(flat_dictionary)
df_features
# for feature in data['features']:
#     print(feature['geometry'])
#     print(feature['properties'])

{'type': ['Point', 'Point', 'Point'], 'coordinates': [[66.0, 33.0], [15.0, 49.75], [15.0, 49.75]], 'urlpubtimedate': ['2018-02-01T17:45:00Z', '2018-02-01T17:45:00Z', '2018-02-01T17:45:00Z'], 'name': ['Afghanistan', 'Czech Republic', 'Czech Republic'], 'urltone': [0.8, 0.8, 0.8], 'url': ['https://www.parlamentnilisty.cz/profily/Ing-Karla-Slechtova-50638/clanek/Ucast-na-misi-v-Afghanistanu-je-skutecne-dulezita-83894', 'https://www.parlamentnilisty.cz/profily/Ing-Karla-Slechtova-50638/clanek/Ucast-na-misi-v-Afghanistanu-je-skutecne-dulezita-83894', 'https://www.parlamentnilisty.cz/profily/Ing-Karla-Slechtova-50638/clanek/Ucast-na-misi-v-Afghanistanu-je-skutecne-dulezita-83894'], 'mentionedthemes': [';TAX_FNCACT_DEPUTY;MILITARY;TAX_MILITARY_TITLE_SOLDIERS;TAX_FNCACT_SOLDIERS;TAX_FNCACT_TROOPS;TAX_ETHNICITY_CZECHS;TAX_WORLDLANGUAGES_CZECHS;TAX_FNCACT_DIRECTOR;TAX_FNCACT_MINISTER;TAX_FNCACT_MINISTER;TAX_ETHNICITY_CZECH;TAX_WORLDLANGUAGES_CZECH;UNGP_FORESTS_RIVERS_OCEANS;', ';TAX_FNCACT_DEPUT

,coordinates,mentionedthemes,name,type,url,urlpubtimedate,urltone
0,"[66.0, 33.0]",;TAX_FNCACT_DEPUTY;MILITARY;TAX_MILITARY_TITLE...,Afghanistan,Point,https://www.parlamentnilisty.cz/profily/Ing-Ka...,2018-02-01T17:45:00Z,0.8
1,"[15.0, 49.75]",;TAX_FNCACT_DEPUTY;MILITARY;TAX_MILITARY_TITLE...,Czech Republic,Point,https://www.parlamentnilisty.cz/profily/Ing-Ka...,2018-02-01T17:45:00Z,0.8
2,"[15.0, 49.75]",;TAX_FNCACT_DEPUTY;MILITARY;TAX_MILITARY_TITLE...,Czech Republic,Point,https://www.parlamentnilisty.cz/profily/Ing-Ka...,2018-02-01T17:45:00Z,0.8
